In [1]:
import argparse
import os
import time
import math
import numpy as np
import random
import sys
import json

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

from utils import to_gpu, Corpus, batchify
from model_glove import Seq2Seq2Decoder, Seq2Seq, MLP_G, MLP_Classify
from models import MLP_D
import shutil
import pdb


In [2]:
print(sys.version)

3.6.7 |Anaconda, Inc.| (default, Oct 23 2018, 19:16:44) 
[GCC 7.3.0]


In [3]:
parser = argparse.ArgumentParser(description='ARAE for Yelp transfer')
# Path Arguments
parser.add_argument('--data_path', type=str, required=True,
                    help='location of the data corpus')
parser.add_argument('--outf', type=str, default="_output",
                    help='output directory name')
parser.add_argument('--load_vocab', type=str, default="",
                    help='path to load vocabulary from')

# Data Processing Arguments
parser.add_argument('--vocab_size', type=int, default=30000,
                    help='cut vocabulary down to this size '
                         '(most frequently seen words in train)')
parser.add_argument('--maxlen', type=int, default=25,
                    help='maximum sentence length')
parser.add_argument('--lowercase', dest='lowercase', action='store_true',
                    help='lowercase all text')
parser.add_argument('--no-lowercase', dest='lowercase', action='store_true',
                    help='not lowercase all text')
parser.set_defaults(lowercase=True)

# Model Arguments
parser.add_argument('--emsize', type=int, default=300,
                    help='size of word embeddings')
parser.add_argument('--nhidden', type=int, default=300,
                    help='number of hidden units per layer')
parser.add_argument('--nlayers', type=int, default=1,
                    help='number of layers')
parser.add_argument('--noise_r', type=float, default=0.1,
                    help='stdev of noise for autoencoder (regularizer)')
parser.add_argument('--noise_anneal', type=float, default=0.9995,
                    help='anneal noise_r exponentially by this'
                         'every 100 iterations')
parser.add_argument('--hidden_init', action='store_true',
                    help="initialize decoder hidden state with encoder's")
parser.add_argument('--arch_g', type=str, default='300-300',
                    help='generator architecture (MLP)')
parser.add_argument('--arch_d', type=str, default='300-300',
                    help='critic/discriminator architecture (MLP)')
parser.add_argument('--arch_classify', type=str, default='300-300',
                    help='classifier architecture')
parser.add_argument('--z_size', type=int, default=32,
                    help='dimension of random noise z to feed into generator')
parser.add_argument('--temp', type=float, default=1,
                    help='softmax temperature (lower --> more discrete)')
parser.add_argument('--dropout', type=float, default=0.0,
                    help='dropout applied to layers (0 = no dropout)')

# Training Arguments
parser.add_argument('--epochs', type=int, default=25,
                    help='maximum number of epochs')
parser.add_argument('--batch_size', type=int, default=64, metavar='N',
                    help='batch size')
parser.add_argument('--niters_ae', type=int, default=1,
                    help='number of autoencoder iterations in training')
parser.add_argument('--niters_gan_d', type=int, default=5,
                    help='number of discriminator iterations in training')
parser.add_argument('--niters_gan_g', type=int, default=1,
                    help='number of generator iterations in training')
parser.add_argument('--niters_gan_ae', type=int, default=1,
                    help='number of gan-into-ae iterations in training')
parser.add_argument('--niters_gan_schedule', type=str, default='',
                    help='epoch counts to increase number of GAN training '
                         ' iterations (increment by 1 each time)')
parser.add_argument('--lr_ae', type=float, default=1,
                    help='autoencoder learning rate')
parser.add_argument('--lr_gan_g', type=float, default=1e-04,
                    help='generator learning rate')
parser.add_argument('--lr_gan_d', type=float, default=1e-04,
                    help='critic/discriminator learning rate')
parser.add_argument('--lr_classify', type=float, default=1e-04,
                    help='classifier learning rate')
parser.add_argument('--beta1', type=float, default=0.5,
                    help='beta1 for adam. default=0.5')
parser.add_argument('--clip', type=float, default=1,
                    help='gradient clipping, max norm')
parser.add_argument('--gan_gp_lambda', type=float, default=0.1,
                    help='WGAN GP penalty lambda')
parser.add_argument('--grad_lambda', type=float, default=0.01,
                    help='WGAN into AE lambda')
parser.add_argument('--lambda_class', type=float, default=1,
                    help='lambda on classifier')

# Evaluation Arguments
parser.add_argument('--sample', action='store_true',
                    help='sample when decoding for generation')
parser.add_argument('--log_interval', type=int, default=1,
                    help='interval to log autoencoder training results')

# Other
parser.add_argument('--seed', type=int, default=1111,
                    help='random seed')
parser.add_argument('--cuda', dest='cuda', action='store_true',
                    help='use CUDA')
parser.add_argument('--no-cuda', dest='cuda', action='store_true',
                    help='not using CUDA')
parser.set_defaults(cuda=True)
parser.add_argument('--device_id', type=str, default='0')

args = parser.parse_args(['--data_path', 'data_yelp'])#Set datapath heree!!!
if(args.outf=="_output_glove"):
    args.outf = args.data_path + args.outf
print(vars(args))

os.environ['CUDA_VISIBLE_DEVICES'] = args.device_id

# make output directory if it doesn't already exist
if os.path.isdir(args.outf):
    shutil.rmtree(args.outf)
os.makedirs(args.outf)

# Set the random seed manually for reproducibility.
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
if torch.cuda.is_available():
    if not args.cuda:
        print("WARNING: You have a CUDA device, "
              "so you should probably run with --cuda")
    else:
        torch.cuda.manual_seed(args.seed)



{'data_path': 'data_yelp', 'outf': '_output', 'load_vocab': '', 'vocab_size': 30000, 'maxlen': 25, 'lowercase': True, 'emsize': 300, 'nhidden': 300, 'nlayers': 1, 'noise_r': 0.1, 'noise_anneal': 0.9995, 'hidden_init': False, 'arch_g': '300-300', 'arch_d': '300-300', 'arch_classify': '300-300', 'z_size': 32, 'temp': 1, 'dropout': 0.0, 'epochs': 25, 'batch_size': 64, 'niters_ae': 1, 'niters_gan_d': 5, 'niters_gan_g': 1, 'niters_gan_ae': 1, 'niters_gan_schedule': '', 'lr_ae': 1, 'lr_gan_g': 0.0001, 'lr_gan_d': 0.0001, 'lr_classify': 0.0001, 'beta1': 0.5, 'clip': 1, 'gan_gp_lambda': 0.1, 'grad_lambda': 0.01, 'lambda_class': 1, 'sample': False, 'log_interval': 1, 'seed': 1111, 'cuda': True, 'device_id': '0'}


In [4]:
label_ids = {"pos": 1, "neg": 0}
id2label = {1: "pos", 0: "neg"}

# (Path to textfile, Name, Use4Vocab)
datafiles = [(os.path.join(args.data_path, "valid1.txt"), "valid1", False),
             (os.path.join(args.data_path, "valid2.txt"), "valid2", False),
             (os.path.join(args.data_path, "train1.txt"), "train1", True),
             (os.path.join(args.data_path, "train2.txt"), "train2", True)]
vocabdict = None
if args.load_vocab != "":
    vocabdict = json.load(args.vocab)
    vocabdict = {k: int(v) for k, v in vocabdict.items()}
corpus = Corpus(datafiles,
                maxlen=args.maxlen,
                vocab_size=args.vocab_size,
                lowercase=args.lowercase,
                vocab=vocabdict)

# dumping vocabulary
with open('{}/vocab.json'.format(args.outf), 'w') as f:
    json.dump(corpus.dictionary.word2idx, f) #Need to edit this if we are using GLOVE

# save arguments
ntokens = len(corpus.dictionary.word2idx)
print("Vocabulary Size: {}".format(ntokens))
args.ntokens = ntokens
with open('{}/args.json'.format(args.outf), 'w') as f:
    json.dump(vars(args), f)
with open("{}/log.txt".format(args.outf), 'w') as f:
    f.write(str(vars(args)))
    f.write("\n\n")

    
eval_batch_size = 100
test1_data = batchify(corpus.data['valid1'], eval_batch_size, shuffle=False)
test2_data = batchify(corpus.data['valid2'], eval_batch_size, shuffle=False)
train1_data = batchify(corpus.data['train1'], args.batch_size, shuffle=True)
train2_data = batchify(corpus.data['train2'], args.batch_size, shuffle=True)

print("Loaded data!")
 


Original vocab 9599; Pruned to 9603
Number of sentences dropped from data_yelp/valid1.txt: 0 out of 38205 total
Number of sentences dropped from data_yelp/valid2.txt: 0 out of 25278 total
Number of sentences dropped from data_yelp/train1.txt: 0 out of 267314 total
Number of sentences dropped from data_yelp/train2.txt: 0 out of 176787 total
Vocabulary Size: 9603
382 batches
252 batches
4176 batches
2762 batches
Loaded data!


In [10]:
!pip install bcolz
!pip install pickle
import bcolz
import pickle
vectors = bcolz.open(f'6B.300.dat')[:]
words = pickle.load(open(f'6B.300words.pkl', 'rb'))
word2idx = pickle.load(open(f'6B.300_idx.pkl', 'rb'))

glove = {w: vectors[word2idx[w]] for w in words}

matrix_len = ntokens
weights_matrix = np.zeros((matrix_len, 300))
words_found = 0
target_vocab = corpus.dictionary.word2idx

for key, value in target_vocab.items():
    try: 
        weights_matrix[value] = glove[key]
        words_found += 1
    except KeyError:
        weights_matrix[value] = np.random.normal(scale=0.6, size=(300, ))
weights_matrix = torch.from_numpy(weights_matrix).float().cuda()

  Could not find a version that satisfies the requirement pickle (from versions: )
No matching distribution found for pickle


#Build model#

Component to experiment with

In [11]:
class CNN_D(torch.nn.Module):

    def __init__(self, ninput, noutput, layers, activation=nn.LeakyReLU(0.2), gpu=False):
        #pdb.set_trace()
        super(CNN_D, self).__init__()
        self.ninput = ninput
        self.noutput = noutput
        self.conv1 = nn.Conv2d(self.ninput, 1, kernel_size=4,\
                               stride=1, padding=2, bias=True)
        self.activation = activation
        self.dropout = nn.Dropout(0.3)
        #pdb.set_trace()
        # flatten before feeding into fully connected
        self.fc1 = nn.Linear(256, noutput, bias=True)  # calculate size here,1 bias = True)
        #256 is palce holder
    def forward(self, x):
        #pdb.set_trace()
        # get last item
        x = x.unsqueeze(-1)
        x = x.unsqueeze(-1)
        #print(x.shape)
        #pdb.set_trace()
        x=self.conv1(x)
        x=self.dropout(self.activation(x))
        x=x.view(-1)
        #print(self.noutput,'after',x.shape)
        logits=self.fc1(x)
        #print(logits.shape)
        #pdb.set_trace()
        return logits
# print("skip this pls")

In [14]:
ntokens = len(corpus.dictionary.word2idx)
autoencoder = Seq2Seq2Decoder(emsize=args.emsize,\
                              nhidden=args.nhidden,\
                              ntokens=ntokens,\
                              nlayers=args.nlayers,\
                              weight_matrix = weights_matrix,\
                              noise_r=args.noise_r,\
                              hidden_init=args.hidden_init,\
                              dropout=args.dropout,\
                              gpu=args.cuda)

gan_gen = MLP_G(ninput=args.z_size, noutput=args.nhidden, layers=args.arch_g)
gan_disc = MLP_D(ninput=args.nhidden, noutput=1, layers=args.arch_d)
print(gan_disc)
#pdb.set_trace()
classifier = MLP_Classify(
    ninput=args.nhidden, noutput=1, layers=args.arch_classify)
g_factor = None

print(autoencoder)
print(gan_gen)
print(gan_disc)
print(classifier)

optimizer_ae = optim.SGD(autoencoder.parameters(), lr=args.lr_ae)
optimizer_gan_g = optim.Adam(gan_gen.parameters(),\
                             lr=args.lr_gan_g,\
                             betas=(args.beta1, 0.999))
optimizer_gan_d = optim.Adam(gan_disc.parameters(),\
                             lr=args.lr_gan_d,\
                             betas=(args.beta1, 0.999))
# classify
optimizer_classify = optim.Adam(classifier.parameters(),\
                                lr=args.lr_classify,\
                                betas=(args.beta1, 0.999))

criterion_ce = nn.CrossEntropyLoss()

if args.cuda:
    autoencoder = autoencoder.cuda()
    gan_gen = gan_gen.cuda()
    gan_disc = gan_disc.cuda()
    classifier = classifier.cuda()
    criterion_ce = criterion_ce.cuda()


MLP_D(
  (layer1): Linear(in_features=300, out_features=300, bias=True)
  (activation1): LeakyReLU(negative_slope=0.2)
  (layer2): Linear(in_features=300, out_features=300, bias=True)
  (bn2): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation2): LeakyReLU(negative_slope=0.2)
  (layer6): Linear(in_features=300, out_features=1, bias=True)
)
Seq2Seq2Decoder(
  (embedding): Embedding(9603, 300)
  (embedding_decoder1): Embedding(9603, 300)
  (embedding_decoder2): Embedding(9603, 300)
  (encoder): LSTM(300, 300, batch_first=True)
  (decoder1): LSTM(600, 300, batch_first=True)
  (decoder2): LSTM(600, 300, batch_first=True)
  (linear): Linear(in_features=300, out_features=9603, bias=True)
)
MLP_G(
  (layer1): Linear(in_features=32, out_features=300, bias=True)
  (bn1): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation1): ReLU()
  (layer2): Linear(in_features=300, out_features=300, bias=True)
  (bn2): Batc

In [15]:
def save_model():
    print("Saving models")
    with open('{}/autoencoder_model.pt'.format(args.outf), 'wb') as f:
        torch.save(autoencoder.state_dict(), f)
    with open('{}/gan_gen_model.pt'.format(args.outf), 'wb') as f:
        torch.save(gan_gen.state_dict(), f)
    with open('{}/gan_disc_model.pt'.format(args.outf), 'wb') as f:
        torch.save(gan_disc.state_dict(), f)


def train_classifier(whichclass, batch):
    classifier.train()
    classifier.zero_grad()

    source, target, lengths = batch
    source = to_gpu(args.cuda, Variable(source))
    labels = to_gpu(args.cuda, Variable(
        torch.zeros(source.size(0)).fill_(whichclass - 1)))

    # Train
    code = autoencoder(0, source, lengths, noise=False,
                       encode_only=True).detach()
    scores = classifier(code)
    classify_loss = F.binary_cross_entropy(scores.squeeze(1), labels)
    classify_loss.backward()
    optimizer_classify.step()
    classify_loss = classify_loss.cpu().data[0]

    pred = scores.data.round().squeeze(1)
    accuracy = pred.eq(labels.data).float().mean()

    return classify_loss, accuracy


def grad_hook_cla(grad):
    return grad * args.lambda_class


def classifier_regularize(whichclass, batch):
    autoencoder.train()
    autoencoder.zero_grad()

    source, target, lengths = batch
    source = to_gpu(args.cuda, Variable(source))
    target = to_gpu(args.cuda, Variable(target))
    flippedclass = abs(2 - whichclass)
    labels = to_gpu(args.cuda, Variable(
        torch.zeros(source.size(0)).fill_(flippedclass)))

    # Train
    code = autoencoder(0, source, lengths, noise=False, encode_only=True)
    code.register_hook(grad_hook_cla)
    scores = classifier(code)
    classify_reg_loss = F.binary_cross_entropy(scores.squeeze(1), labels)
    classify_reg_loss.backward()

    torch.nn.utils.clip_grad_norm_(autoencoder.parameters(), args.clip)
    optimizer_ae.step()

    return classify_reg_loss


def evaluate_autoencoder(whichdecoder, data_source, epoch):
    # Turn on evaluation mode which disables dropout.
    autoencoder.eval()
    total_loss = 0
    ntokens = len(corpus.dictionary.word2idx)
    all_accuracies = 0
    bcnt = 0
    for i, batch in enumerate(data_source):
        source, target, lengths = batch
        source = to_gpu(args.cuda, Variable(source, volatile=True))
        target = to_gpu(args.cuda, Variable(target, volatile=True))

        mask = target.gt(0)
        masked_target = target.masked_select(mask)
        # examples x ntokens
        output_mask = mask.unsqueeze(1).expand(mask.size(0), ntokens)

        hidden = autoencoder(0, source, lengths, noise=False, encode_only=True)

        # output: batch x seq_len x ntokens
        if whichdecoder == 1:
            output = autoencoder(1, source, lengths, noise=False)
            flattened_output = output.view(-1, ntokens)
            masked_output = \
                flattened_output.masked_select(output_mask).view(-1, ntokens)
            # accuracy
            max_vals1, max_indices1 = torch.max(masked_output, 1)
            all_accuracies += \
                torch.mean(max_indices1.eq(masked_target).float()).data[0]

            max_values1, max_indices1 = torch.max(output, 2)
            max_indices2 = autoencoder.generate(2, hidden, maxlen=50)
        else:
            output = autoencoder(2, source, lengths, noise=False)
            flattened_output = output.view(-1, ntokens)
            masked_output = \
                flattened_output.masked_select(output_mask).view(-1, ntokens)
            # accuracy
            max_vals2, max_indices2 = torch.max(masked_output, 1)
            all_accuracies += \
                torch.mean(max_indices2.eq(masked_target).float()).data[0]

            max_values2, max_indices2 = torch.max(output, 2)
            max_indices1 = autoencoder.generate(1, hidden, maxlen=50)

        total_loss += criterion_ce(masked_output /
                                   args.temp, masked_target).data
        bcnt += 1

        aeoutf_from = "{}/{}_output_decoder_{}_from.txt".format(
            args.outf, epoch, whichdecoder)
        aeoutf_tran = "{}/{}_output_decoder_{}_tran.txt".format(
            args.outf, epoch, whichdecoder)
        with open(aeoutf_from, 'w') as f_from, open(aeoutf_tran, 'w') as f_trans:
            max_indices1 = \
                max_indices1.view(output.size(0), -1).data.cpu().numpy()
            max_indices2 = \
                max_indices2.view(output.size(0), -1).data.cpu().numpy()
            target = target.view(output.size(0), -1).data.cpu().numpy()
            tran_indices = max_indices2 if whichdecoder == 1 else max_indices1
            for t, tran_idx in zip(target, tran_indices):
                # real sentence
                chars = " ".join([corpus.dictionary.idx2word[x] for x in t])
                f_from.write(chars)
                f_from.write("\n")
                # transfer sentence
                chars = " ".join([corpus.dictionary.idx2word[x]
                                  for x in tran_idx])
                f_trans.write(chars)
                f_trans.write("\n")

    return total_loss[0] / len(data_source), all_accuracies / bcnt


def evaluate_generator(whichdecoder, noise, epoch):
    gan_gen.eval()
    autoencoder.eval()

    # generate from fixed random noise
    fake_hidden = gan_gen(noise)
    max_indices = \
        autoencoder.generate(whichdecoder, fake_hidden,
                             maxlen=50, sample=args.sample)

    with open("%s/%s_generated%d.txt" % (args.outf, epoch, whichdecoder), "w") as f:
        max_indices = max_indices.data.cpu().numpy()
        for idx in max_indices:
            # generated sentence
            #Use intermediate for glove here
            words = [corpus.dictionary.idx2word[x] for x in idx]
            # truncate sentences to first occurrence of <eos>
            truncated_sent = []
            for w in words:
                if w != '<eos>':
                    truncated_sent.append(w)
                else:
                    break
            chars = " ".join(truncated_sent)
            f.write(chars)
            #print("word length is:", len(words))
            #print('generated output:\n', chars)
            f.write("\n")


def train_ae(whichdecoder, batch, total_loss_ae, start_time, i):
    autoencoder.train()
    optimizer_ae.zero_grad()

    source, target, lengths = batch
    source = to_gpu(args.cuda, Variable(source))
    target = to_gpu(args.cuda, Variable(target))

    mask = target.gt(0)
    masked_target = target.masked_select(mask)
    output_mask = mask.unsqueeze(1).expand(mask.size(0), ntokens)
    output = autoencoder(whichdecoder, source, lengths, noise=True)
    flat_output = output.view(-1, ntokens)
    masked_output = flat_output.masked_select(output_mask).view(-1, ntokens)
    loss = criterion_ce(masked_output / args.temp, masked_target)
    loss.backward()

    # `clip_grad_norm` to prevent exploding gradient in RNNs / LSTMs
    torch.nn.utils.clip_grad_norm_(autoencoder.parameters(), args.clip)
    optimizer_ae.step()

    total_loss_ae += loss.data

    accuracy = None
    if i % args.log_interval == 0 and i > 0:
        probs = F.softmax(masked_output, dim=-1)
        max_vals, max_indices = torch.max(probs, 1)
        accuracy = torch.mean(max_indices.eq(masked_target).float()).data[0]
        cur_loss = total_loss_ae[0] / args.log_interval
        elapsed = time.time() - start_time
        print('| epoch {:3d} | {:5d}/{:5d} batches | ms/batch {:5.2f} | '
              'loss {:5.2f} | ppl {:8.2f} | acc {:8.2f}'
              .format(epoch, i, len(train1_data),
                      elapsed * 1000 / args.log_interval,
                      cur_loss, math.exp(cur_loss), accuracy))

        with open("{}/log.txt".format(args.outf), 'a') as f:
            f.write('| epoch {:3d} | {:5d}/{:5d} batches | ms/batch {:5.2f} | '
                    'loss {:5.2f} | ppl {:8.2f} | acc {:8.2f}\n'.
                    format(epoch, i, len(train1_data),
                           elapsed * 1000 / args.log_interval,
                           cur_loss, math.exp(cur_loss), accuracy))

        total_loss_ae = 0
        start_time = time.time()

    return total_loss_ae, start_time


def train_gan_g():
    gan_gen.train()
    gan_gen.zero_grad()

    noise = to_gpu(args.cuda,Variable(torch.ones(args.batch_size, args.z_size)))
    noise.data.normal_(0, 1)
    fake_hidden = gan_gen(noise)
    errG = gan_disc(fake_hidden)
    errG.backward(one)
    optimizer_gan_g.step()

    return errG


def grad_hook(grad):
    return grad * args.grad_lambda


''' Steal from https://github.com/caogang/wgan-gp/blob/master/gan_cifar10.py '''


def calc_gradient_penalty(netD, real_data, fake_data):
    bsz = real_data.size(0)
    alpha = torch.rand(bsz, 1)
    alpha = alpha.expand(bsz, real_data.size(1))  # only works for 2D XXX
    alpha = alpha.cuda()
    interpolates = alpha * real_data + ((1 - alpha) * fake_data)
    interpolates = Variable(interpolates, requires_grad=True)
    disc_interpolates = netD(interpolates)

    gradients = torch.autograd.grad(outputs=disc_interpolates, inputs=interpolates,\
                                    grad_outputs=torch.ones(\
                                        disc_interpolates.size()).cuda(),\
                                    create_graph=True, retain_graph=True, only_inputs=True)[0]
    gradients = gradients.view(gradients.size(0), -1)

    gradient_penalty = ((gradients.norm(2, dim=1) - 1)
                        ** 2).mean() * args.gan_gp_lambda
    return gradient_penalty


def train_gan_d(whichdecoder, batch):
    gan_disc.train()
    optimizer_gan_d.zero_grad()

    # positive samples ----------------------------
    # generate real codes
    source, target, lengths = batch
    source = to_gpu(args.cuda, Variable(source))
    target = to_gpu(args.cuda, Variable(target))

    # batch_size x nhidden
    real_hidden = autoencoder(whichdecoder, source,lengths, noise=False, encode_only=True)

    # loss / backprop
    errD_real = gan_disc(real_hidden)
    errD_real.backward(one)

    # negative samples ----------------------------
    # generate fake codes
    noise = to_gpu(args.cuda,
                Variable(torch.ones(args.batch_size, args.z_size)))
    noise.data.normal_(0, 1)

    # loss / backprop
    fake_hidden = gan_gen(noise)
    errD_fake = gan_disc(fake_hidden.detach())
    errD_fake.backward(mone)

    # gradient penalty
    gradient_penalty = calc_gradient_penalty(gan_disc, real_hidden.data, fake_hidden.data)
    gradient_penalty.backward()

    optimizer_gan_d.step()
    errD = -(errD_real.unsqueeze(0) - errD_fake.unsqueeze(0))

    return errD, errD_real, errD_fake


def train_gan_d_into_ae(whichdecoder, batch):
    autoencoder.train()
    optimizer_ae.zero_grad()

    source, target, lengths = batch
    source = to_gpu(args.cuda, Variable(source))
    target = to_gpu(args.cuda, Variable(target))
    real_hidden = autoencoder(whichdecoder, source, lengths, noise=False, encode_only=True)
    real_hidden.register_hook(grad_hook)
    errD_real = gan_disc(real_hidden)
    errD_real.backward(mone)
    torch.nn.utils.clip_grad_norm_(autoencoder.parameters(), args.clip)

    optimizer_ae.step()

    return errD_real

In [ ]:
print("Training...")
with open("{}/log.txt".format(args.outf), 'a') as f:
    f.write('Training...\n')

# schedule of increasing GAN training loops
if args.niters_gan_schedule != "":
    gan_schedule = [int(x) for x in args.niters_gan_schedule.split("-")]
else:
    gan_schedule = []
niter_gan = 25

fixed_noise = to_gpu(args.cuda,Variable(torch.ones(args.batch_size, args.z_size)))
fixed_noise.data.normal_(0, 1)
one = Variable(torch.tensor(1.0).cuda())#to_gpu(args.cuda, torch.FloatTensor([1]))
mone = Variable(torch.tensor(-1.0).cuda())#one * -1
#one = Variable(one, requires_grad=True).cuda()#torch.tensor(1.0, dtype=torch.float64,device=torch.device('cuda:0'))
#mone = #Variable(mone, requires_grad=True).cuda()
#mone = torch.tensor(-1.0, dtype=torch.float64,device=torch.device('cuda:0'))



for epoch in range(1, args.epochs + 1):
    # update gan training schedule
    if epoch in gan_schedule:
        niter_gan += 1
        print("GAN training loop schedule increased to {}".format(niter_gan))
        with open("{}/log.txt".format(args.outf), 'a') as f:
            f.write("GAN training loop schedule increased to {}\n".
                    format(niter_gan))

    total_loss_ae1 = 0
    total_loss_ae2 = 0
    classify_loss = 0
    epoch_start_time = time.time()
    start_time = time.time()
    niter = 0
    niter_global = 1

    # loop through all batches in training data
    while niter < len(train1_data) and niter < len(train2_data):

        # train autoencoder ----------------------------
        for i in range(args.niters_ae):
            if niter == len(train1_data):
                break  # end of epoch
            total_loss_ae1, start_time = train_ae(1, train1_data[niter], total_loss_ae1, start_time, niter)
            total_loss_ae2, _ = train_ae(2, train2_data[niter], total_loss_ae2, start_time, niter)

            # train classifier ----------------------------
            classify_loss1, classify_acc1 = train_classifier(
                1, train1_data[niter])
            classify_loss2, classify_acc2 = train_classifier(
                2, train2_data[niter])
            classify_loss = (classify_loss1 + classify_loss2) / 2
            classify_acc = (classify_acc1 + classify_acc2) / 2
            # reverse to autoencoder
            classifier_regularize(1, train1_data[niter])
            classifier_regularize(2, train2_data[niter])

            niter += 1

        # train gan ----------------------------------
        for k in range(niter_gan):

            # train discriminator/critic
            for i in range(args.niters_gan_d):
                # feed a seen sample within this epoch; good for early training
                if i % 2 == 0:
                    batch = train1_data[
                        random.randint(0, len(train1_data) - 1)]
                    whichdecoder = 1
                else:
                    batch = train2_data[
                        random.randint(0, len(train2_data) - 1)]
                    whichdecoder = 2
                errD, errD_real, errD_fake = train_gan_d(whichdecoder, batch)

            # train generator
            for i in range(args.niters_gan_g):
                errG = train_gan_g()

            # train autoencoder from d
            for i in range(args.niters_gan_ae):
                if i % 2 == 0:
                    batch = train1_data[
                        random.randint(0, len(train1_data) - 1)]
                    whichdecoder = 1
                else:
                    batch = train2_data[
                        random.randint(0, len(train2_data) - 1)]
                    whichdecoder = 2
                errD_= train_gan_d_into_ae(whichdecoder, batch)

        niter_global += 1
        if niter_global % 100 >= 0:
            print('[%d/%d][%d/%d] Loss_D: %.4f (Loss_D_real: %.4f '
                  'Loss_D_fake: %.4f) Loss_G: %.4f'
                  % (epoch, args.epochs, niter, len(train1_data),
                     errD.data[0], errD_real.data[0],
                     errD_fake.data[0], errG.data[0]))
            print("Classify loss: {:5.2f} | Classify accuracy: {:3.3f}\n".format(
                classify_loss, classify_acc))
            with open("{}/log.txt".format(args.outf), 'a') as f:
                f.write('[%d/%d][%d/%d] Loss_D: %.4f (Loss_D_real: %.4f '
                        'Loss_D_fake: %.4f) Loss_G: %.4f\n'
                        % (epoch, args.epochs, niter, len(train1_data),
                           errD.data[0], errD_real.data[0],
                           errD_fake.data[0], errG.data[0]))
                f.write("Classify loss: {:5.2f} | Classify accuracy: {:3.3f}\n".format(
                        classify_loss, classify_acc))

            # exponentially decaying noise on autoencoder
            autoencoder.noise_r = \
                autoencoder.noise_r * args.noise_anneal

    # end of epoch ----------------------------
    # evaluation
    test_loss, accuracy = evaluate_autoencoder(1, test1_data[:1000], epoch)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | test loss {:5.2f} | '
          'test ppl {:5.2f} | acc {:3.3f}'.
          format(epoch, (time.time() - epoch_start_time),
                 test_loss, math.exp(test_loss), accuracy))
    print('-' * 89)
    with open("{}/log.txt".format(args.outf), 'a') as f:
        f.write('-' * 89)
        f.write('\n| end of epoch {:3d} | time: {:5.2f}s | test loss {:5.2f} |'
                ' test ppl {:5.2f} | acc {:3.3f}\n'.
                format(epoch, (time.time() - epoch_start_time),
                       test_loss, math.exp(test_loss), accuracy))
        f.write('-' * 89)
        f.write('\n')

    test_loss, accuracy = evaluate_autoencoder(2, test2_data[:1000], epoch)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | test loss {:5.2f} | '
          'test ppl {:5.2f} | acc {:3.3f}'.
          format(epoch, (time.time() - epoch_start_time),
                 test_loss, math.exp(test_loss), accuracy))
    print('-' * 89)
    with open("{}/log.txt".format(args.outf), 'a') as f:
        f.write('-' * 89)
        f.write('\n| end of epoch {:3d} | time: {:5.2f}s | test loss {:5.2f} |'
                ' test ppl {:5.2f} | acc {:3.3f}\n'.
                format(epoch, (time.time() - epoch_start_time),
                       test_loss, math.exp(test_loss), accuracy))
        f.write('-' * 89)
        f.write('\n')

    evaluate_generator(1, fixed_noise, "end_of_epoch_{}".format(epoch))
    evaluate_generator(2, fixed_noise, "end_of_epoch_{}".format(epoch))

    # shuffle between epochs
    train1_data = batchify(
        corpus.data['train1'], args.batch_size, shuffle=True)
    train2_data = batchify(
        corpus.data['train2'], args.batch_size, shuffle=True)


test_loss, accuracy = evaluate_autoencoder(1, test1_data, epoch + 1)
print('-' * 89)
print('| end of epoch {:3d} | time: {:5.2f}s | test loss {:5.2f} | '
      'test ppl {:5.2f} | acc {:3.3f}'.
      format(epoch, (time.time() - epoch_start_time),
             test_loss, math.exp(test_loss), accuracy))
print('-' * 89)
with open("{}/log.txt".format(args.outf), 'a') as f:
    f.write('-' * 89)
    f.write('\n| end of epoch {:3d} | time: {:5.2f}s | test loss {:5.2f} |'
            ' test ppl {:5.2f} | acc {:3.3f}\n'.
            format(epoch, (time.time() - epoch_start_time),
                   test_loss, math.exp(test_loss), accuracy))
    f.write('-' * 89)
    f.write('\n')

test_loss, accuracy = evaluate_autoencoder(2, test2_data, epoch + 1)
print('-' * 89)
print('| end of epoch {:3d} | time: {:5.2f}s | test loss {:5.2f} | '
      'test ppl {:5.2f} | acc {:3.3f}'.
      format(epoch, (time.time() - epoch_start_time),
             test_loss, math.exp(test_loss), accuracy))
print('-' * 89)
with open("{}/log.txt".format(args.outf), 'a') as f:
    f.write('-' * 89)
    f.write('\n| end of epoch {:3d} | time: {:5.2f}s | test loss {:5.2f} |'
            ' test ppl {:5.2f} | acc {:3.3f}\n'.
            format(epoch, (time.time() - epoch_start_time),
                   test_loss, math.exp(test_loss), accuracy))
    f.write('-' * 89)
    f.write('\n')

Training...


/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:27: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:99: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:100: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:107: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:108: UserWarning: invalid index of a 0-dim t

[1/25][1/4176] Loss_D: 0.0517 (Loss_D_real: -0.0505 Loss_D_fake: 0.0012) Loss_G: 0.0014
Classify loss:  0.69 | Classify accuracy: 0.414

| epoch   1 |     1/ 4176 batches | ms/batch 2567.37 | loss 18.17 | ppl 77943403.90 | acc     0.10
| epoch   1 |     1/ 4176 batches | ms/batch 42.85 | loss 18.11 | ppl 73438962.68 | acc     0.10
[1/25][2/4176] Loss_D: 0.1501 (Loss_D_real: -0.1509 Loss_D_fake: -0.0007) Loss_G: -0.0007
Classify loss:  0.69 | Classify accuracy: 0.500

| epoch   1 |     2/ 4176 batches | ms/batch 2321.51 | loss  8.76 | ppl  6375.10 | acc     0.10
| epoch   1 |     2/ 4176 batches | ms/batch 45.65 | loss  8.66 | ppl  5744.85 | acc     0.11
[1/25][3/4176] Loss_D: 0.2844 (Loss_D_real: -0.2951 Loss_D_fake: -0.0107) Loss_G: -0.0233
Classify loss:  0.69 | Classify accuracy: 0.461

| epoch   1 |     3/ 4176 batches | ms/batch 2345.89 | loss  8.40 | ppl  4454.69 | acc     0.10
| epoch   1 |     3/ 4176 batches | ms/batch 44.43 | loss  8.27 | ppl  3897.03 | acc     0.09
[1/25][4/

[1/25][26/4176] Loss_D: 2.1238 (Loss_D_real: -1.9877 Loss_D_fake: 0.1360) Loss_G: 0.3691
Classify loss:  0.69 | Classify accuracy: 0.672

| epoch   1 |    26/ 4176 batches | ms/batch 2352.41 | loss  5.89 | ppl   360.61 | acc     0.10
| epoch   1 |    26/ 4176 batches | ms/batch 45.65 | loss  6.67 | ppl   785.74 | acc     0.12
[1/25][27/4176] Loss_D: 2.7255 (Loss_D_real: -2.0016 Loss_D_fake: 0.7239) Loss_G: 0.7496
Classify loss:  0.69 | Classify accuracy: 0.609

| epoch   1 |    27/ 4176 batches | ms/batch 2373.63 | loss  6.45 | ppl   635.57 | acc     0.13
| epoch   1 |    27/ 4176 batches | ms/batch 42.58 | loss  6.27 | ppl   529.08 | acc     0.10
[1/25][28/4176] Loss_D: 1.6480 (Loss_D_real: -2.0675 Loss_D_fake: -0.4195) Loss_G: -0.1118
Classify loss:  0.69 | Classify accuracy: 0.648

| epoch   1 |    28/ 4176 batches | ms/batch 2306.63 | loss  6.05 | ppl   423.14 | acc     0.15
| epoch   1 |    28/ 4176 batches | ms/batch 46.41 | loss  7.00 | ppl  1096.61 | acc     0.10
[1/25][29/4176